In [1]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [2]:
try:
    with driver.session() as session:
        result = session.run("RETURN 1 AS test")
    print("Connection successful")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful


In [3]:
import os
with open('./pathCSV.txt','r') as f:
    path = f.read()
os.path.exists(path)

True

In [4]:
business_path = "business.csv"
review_path = "review.csv"
# tip_path = "yelp_academic_dataset_tip.json"
user_path = "users.csv"

business_path, review_path, user_path = list(map(lambda y: path + '/' + y,[business_path, review_path, user_path]))
# business_path, review_path, user_path = list(map(lambda y:'"'+ path + '/' + y+'"',[business_path, review_path, user_path]))
print(business_path)

../filteredCSV/business.csv


In [ ]:
def import_csv(tx, csv_file_path, query):
    with open(csv_file_path, 'r') as file:
        tx.run(query, file=file)
    # with open(csv_file_path, 'r') as file:
    #     csv_data = file.read()
    # tx.run(query,file=file)
# csv_file_path = "path/to/your/data.csv"

import pandas as pd

# Read CSV into Pandas DataFrame
users = pd.read_csv(business_path)
print(users.columns)

# Function to create nodes and relationships
def create_graph(tx, df):
    for _, row in df.iterrows():
        # query = """CREATE (n:Businses  {name:$name,
        #change this to use SET so that ubusiness ID is only thing that is unique that is tehre
        query = """Merge (n:Businses  {business_id:$business_id})
                SET n.name=$name, 
                    n.address=$address,
                    n.city=$city,
                    n.state=$state,
                    n.latitude=$latitude,
                    n.longitude=$longitude,
                    n.stars=$stars,
                    n.review_count=$review_count,
                    n.hours=$hours,
                    n.attributes=$attributes})
                    With n, split($categories,', ') as cats
                    unwind cats AS cat
                    merge (c:Category {name: cat})
                    Merge (n)-[:IN_CATEGORY]->(c)
                    """

        tx.run(query, business_id=row['business_id'],
                    address=['address'],
                    name=row['name'],
                    city=row['city'],
                    state=row['state'],
                    latitude=row['latitude'],
                    longitude=row['longitude'],
                    stars=row['stars'],
                    review_count=row['review_count'],
                    hours=row['hours'],
                    attributes=row['attributes'],
                    categories=row['categories'])

# Execute the function within a transaction
with driver.session() as session:
    session.execute_write(create_graph, users)

# Close the driver
driver.close()

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')


C:\Users\susha\AppData\Local\Temp\ipykernel_6480\3299428289.py:50: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
def import_csv(tx, csv_file_path, query):
    with open(csv_file_path, 'r') as file:
        tx.run(query, file=file)

import pandas as pd

# Function to create nodes and relationships
def create_graph(tx, users_data):
    # for _, row in users.iterrows():
    query = """UNWIND $users AS row
            MERGE (n:User {user_id: row.user_id})
                SET n.name = row.name,
                    n.review_count = row.review_count,
                    n.yelping_since = row.yelping_since,
                    n.useful = row.useful,
                    n.funny = row.funny,
                    n.cool = row.cool,
                    n.fans = row.fans,
                    n.average_stars = row.average_stars,
                    n.compliment_hot = row.compliment_hot,
                    n.compliment_more = row.compliment_more,
                    n.compliment_profile = row.compliment_profile,
                    n.compliment_cute = row.compliment_cute,
                    n.compliment_list = row.compliment_list,
                    n.compliment_note = row.compliment_note,
                    n.compliment_plain = row.compliment_plain,
                    n.compliment_cool = row.compliment_cool,
                    n.compliment_funny = row.compliment_funny,
                    n.compliment_writer = row.compliment_writer

                WITH n, split(rtrim(row.friends), ',') AS frens
                UNWIND frens AS friend_id
                MERGE (f:User {user_id: friend_id})
                MERGE (n)-[:HAS_FRIEND]->(f)
                """
    tx.run(query,users_data)

# Execute the function within a transaction
chunk_size = 10000
i=1
for chunk in pd.read_csv(user_path, chunksize=chunk_size):
    print(f"chunk {i} being processed")
    users= chunk.to_dict('records')
    print("chunk converted to dicts")
    with driver.session() as session:
        session.execute_write(create_graph, users)
    print("chunk processed")
    i+=1

# Close the driver
driver.close()

chunk 1 being processed
chunk converted to dicts
